In [ ]:

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader


In [ ]:

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


In [ ]:

class EmotionModel:
    def __init__(self):
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        self.model = RobertaForSequenceClassification.from_pretrained('roberta-base')

    def train(self, train_texts, train_labels, val_texts, val_labels, epochs=3, batch_size=16):
        train_dataset = TextDataset(train_texts, train_labels, self.tokenizer)
        val_dataset = TextDataset(val_texts, val_labels, self.tokenizer)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        optimizer = AdamW(self.model.parameters(), lr=1e-5)
        self.model.train()

        for epoch in range(epochs):
            total_loss = 0
            for batch in train_loader:
                optimizer.zero_grad()
                input_ids = batch['input_ids']
                attention_mask = batch['attention_mask']
                labels = batch['label']
                outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            print(f"Epoch {epoch + 1}/{epochs} - Loss: {total_loss / len(train_loader)}")

    def predict(self, text):
        self.model.eval()
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        with torch.no_grad():
            outputs = self.model(input_ids=encoding['input_ids'], attention_mask=encoding['attention_mask'])
        return torch.softmax(outputs.logits, dim=1).numpy().flatten()


In [ ]:

class CollaborativeFiltering:
    def __init__(self, n_components=50):
        self.n_components = n_components
        self.svd = TruncatedSVD(n_components=self.n_components)

    def fit(self, user_item_matrix):
        self.user_factors = self.svd.fit_transform(user_item_matrix)
        self.item_factors = self.svd.components_

    def predict(self, user_index):
        return np.dot(self.user_factors[user_index], self.item_factors)


In [ ]:

class ContentBasedFiltering:
    def __init__(self, emotion_model):
        self.emotion_model = emotion_model

    def predict(self, user_feedback, item_descriptions):
        user_emotion = self.emotion_model.predict(user_feedback)
        item_emotions = np.array([self.emotion_model.predict(desc) for desc in item_descriptions])
        return cosine_similarity(user_emotion.reshape(1, -1), item_emotions).flatten()


In [ ]:

class StackEnsemble:
    def __init__(self):
        self.meta_model = LogisticRegression()

    def fit(self, base_predictions, y_train):
        self.meta_model.fit(base_predictions, y_train)

    def predict(self, base_predictions):
        return self.meta_model.predict_proba(base_predictions)


In [ ]:

class RecommenderSystem:
    def __init__(self, n_components=50):
        self.collaborative_filtering = CollaborativeFiltering(n_components=n_components)
        self.emotion_model = EmotionModel()
        self.content_based_filtering = ContentBasedFiltering(self.emotion_model)
        self.stack_ensemble = StackEnsemble()

    def train_emotion_model(self, train_texts, train_labels, val_texts, val_labels, epochs=3, batch_size=16):
        self.emotion_model.train(train_texts, train_labels, val_texts, val_labels, epochs, batch_size)

    def fit(self, user_item_matrix, user_feedbacks, item_descriptions, y_train):
        self.collaborative_filtering.fit(user_item_matrix)
        collaborative_predictions = [self.collaborative_filtering.predict(i) for i in range(user_item_matrix.shape[0])]
        content_predictions = [self.content_based_filtering.predict(feedback, item_descriptions) for feedback in user_feedbacks]

        base_predictions = np.hstack((collaborative_predictions, content_predictions))
        self.stack_ensemble.fit(base_predictions, y_train)

    def recommend(self, user_index, user_feedback, item_descriptions):
        collaborative_pred = self.collaborative_filtering.predict(user_index)
        content_pred = self.content_based_filtering.predict(user_feedback, item_descriptions)
        base_pred = np.hstack((collaborative_pred, content_pred)).reshape(1, -1)
        final_pred = self.stack_ensemble.predict(base_pred)
        return np.argsort(final_pred.flatten())[::-1]


In [ ]:

# Example usage
if __name__ == "__main__":
    # Simulated data
    user_item_matrix = np.random.rand(10, 50)
    user_feedbacks = ["I feel good about this product.", "I'm not happy with the quality.", "This product is excellent."]
    item_descriptions = ["A great product for daily use.", "Top-notch quality and performance.", "Affordable and reliable."]

    train_texts = ["I love this!", "Not satisfied", "Works well"]
    train_labels = [1, 0, 1]
    val_texts = ["Amazing product", "Disappointing experience"]
    val_labels = [1, 0]
    y_train = np.random.randint(0, 2, size=10)

    recommender = RecommenderSystem()
    recommender.train_emotion_model(train_texts, train_labels, val_texts, val_labels)

    recommender.fit(user_item_matrix, user_feedbacks, item_descriptions, y_train)
    recommendations = recommender.recommend(user_index=0, user_feedback="I feel overwhelmed by the choices.", item_descriptions=item_descriptions)
    print(f"Recommended item indices: {recommendations[:3]}")
